In [1]:
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import loggers as pl_loggers
from models import FocalClassifier
from evaluation import evaluate

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [4]:
interictal_data = torch.load("data/interictal.pt")
preictal_data = torch.load("data/preictal.pt")

In [5]:
patient_ids = list(interictal_data.keys())

In [6]:
# for test_patient in patient_ids:
#     ##----------------------------------------OVERSAMPLING-----------------------------------------------------------------------------##
#     train_patients = [patient for patient in patient_ids if patient != test_patient]
#     train_X = []
#     train_y = []
#     for patient in train_patients:
#         patient_interictal_X = interictal_data[patient]
#         patient_preictal_X = preictal_data[patient]
#         # Oversampling preictal data
#         sample_indexes = np.random.randint(low=0, high=len(patient_preictal_X), size=len(patient_interictal_X))
#         patient_preictal_X = patient_preictal_X[sample_indexes]

#         patient_interictal_y = torch.zeros(patient_interictal_X.shape[0])
#         patient_preictal_y = torch.ones(patient_preictal_X.shape[0])
        

#         patient_train_X = torch.cat([patient_interictal_X, patient_preictal_X])
#         patient_train_y = torch.cat([patient_interictal_y, patient_preictal_y])

#         train_X.append(patient_train_X)
#         train_y.append(patient_train_y)

#     train_X = torch.cat(train_X)
#     train_y = torch.cat(train_y)

#     ##----------------------------------------UNDERSAMPLING-----------------------------------------------------------------------------##
#     # interictal_train_X = torch.cat([patient_data for patient_id, patient_data in interictal_data.items() if patient_id!=test_patient])
#     # preictal_train_X = torch.cat([patient_data for patient_id, patient_data in preictal_data.items() if patient_id!=test_patient])
#     # # Undersample interictal data
#     # interictal_train_X = interictal_train_X[::int(len(interictal_train_X)/len(preictal_train_X))]
#     # train_X = torch.cat([interictal_train_X, preictal_train_X])

#     # interictal_train_y = torch.zeros(interictal_train_X.shape[0])
#     # preictal_train_y = torch.ones(preictal_train_X.shape[0])
#     # train_y = torch.cat([interictal_train_y, preictal_train_y])
#     ##----------------------------------------------------------------------------------------------------------------------------------##

#     train_dataset = TensorDataset(train_X, train_y)
#     train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

    
#     try:
#         model_path = glob(f"experiments/oversampling/meisel/{test_patient}/version_0/checkpoints/*.ckpt")[0]
#     except IndexError:
#         model = MeiselClassifier()
#         tb_logger = pl_loggers.TensorBoardLogger("experiments/oversampling", name=f"meisel/{test_patient}")
#         trainer = pl.Trainer(gpus=1, max_epochs=200, logger=tb_logger)
#         trainer.fit(model, train_dataloader)

# Evaluation

In [11]:
test_results = pd.DataFrame()
for test_patient in patient_ids:
    model_path = glob(f"experiments/focal_loss/meisel/{test_patient}/version_0/checkpoints/*.ckpt")[0]
    print(model_path)
    model = FocalClassifier.load_from_checkpoint(model_path)
    # train_results = pd.DataFrame()
    # for train_patient in [patient for patient in patient_ids if patient != test_patient]:
    #     train_patient_results = evaluate(model, train_patient, integration_windows=[300000, 600000, 900000], thresholds=np.round(np.arange(0.4, 0.7, 0.02),2), timer_duration=450000, detection_interval=60000)
    #     train_patient_results["patient"] = train_patient
    #     train_results = pd.concat([train_results, train_patient_results])
    # best_IoC = 0
    # for i in train_results.index.unique():
    #     integration_window, threshold = i.split("_")
    #     mean_IoC = train_results.loc[i, "IoC"].mean()
    #     if mean_IoC > best_IoC:
    #         best_integration_window = int(integration_window)
    #         best_threshold = float(threshold)
    # test_patient_results = evaluate(model, test_patient, integration_windows=[best_integration_window], thresholds=[best_threshold], timer_duration=450000, detection_interval=60000)
    # test_patient_results = evaluate(model, test_patient, integration_windows=[300000, 600000, 900000], thresholds=np.round(np.arange(0.4, 0.7, 0.02),2), timer_duration=450000, detection_interval=60000)
    test_patient_results = evaluate(model, test_patient, integration_windows=[600000], thresholds=[0.50], timer_duration=3600000, detection_interval=0)
    test_patient_results["patient"] = test_patient
    test_results = pd.concat([test_results, test_patient_results])

experiments/focal_loss/meisel/MSEL_00095/version_0/checkpoints\epoch=199-step=168599.ckpt
experiments/focal_loss/meisel/MSEL_00172/version_0/checkpoints\epoch=199-step=177799.ckpt
experiments/focal_loss/meisel/MSEL_00182/version_0/checkpoints\epoch=199-step=175399.ckpt
experiments/focal_loss/meisel/MSEL_00501/version_0/checkpoints\epoch=199-step=175599.ckpt
experiments/focal_loss/meisel/MSEL_00502/version_0/checkpoints\epoch=199-step=178999.ckpt
experiments/focal_loss/meisel/MSEL_00764/version_0/checkpoints\epoch=199-step=177399.ckpt
experiments/focal_loss/meisel/MSEL_01097/version_0/checkpoints\epoch=199-step=172199.ckpt
experiments/focal_loss/meisel/MSEL_01110-ICU/version_0/checkpoints\epoch=199-step=175799.ckpt
experiments/focal_loss/meisel/MSEL_01462/version_0/checkpoints\epoch=199-step=177399.ckpt
experiments/focal_loss/meisel/MSEL_01550/version_0/checkpoints\epoch=199-step=172399.ckpt
experiments/focal_loss/meisel/MSEL_01575/version_0/checkpoints\epoch=199-step=172999.ckpt
experi

In [14]:
# test_results.to_csv("experiments/result_focal_loss_0.5.csv")

In [78]:
df = pd.read_csv(r"experiments\result_focal_loss_0.5.csv")

In [79]:
df[df["p"]<0.05]

,Unnamed: 0,Integration Window,threshold,n,N,S,Sc,TiW,IoC,p,patient


In [80]:
df

,Unnamed: 0,Integration Window,threshold,n,N,S,Sc,TiW,IoC,p,patient
0,600000_0.5,600000,0.5,0,3,0.0,0.0,0.0,0.0,100,MSEL_00095
1,600000_0.5,600000,0.5,0,10,0.0,0.0,0.0,0.0,100,MSEL_00172
2,600000_0.5,600000,0.5,0,4,0.0,0.0,0.0,0.0,100,MSEL_00182
3,600000_0.5,600000,0.5,0,1,0.0,0.0,0.0,0.0,100,MSEL_00501
4,600000_0.5,600000,0.5,0,6,0.0,0.0,0.0,0.0,100,MSEL_00502
5,600000_0.5,600000,0.5,0,4,0.0,0.0,0.0,0.0,100,MSEL_00764
6,600000_0.5,600000,0.5,0,2,0.0,0.0,0.0,0.0,100,MSEL_01097
7,600000_0.5,600000,0.5,0,9,0.0,0.0,0.0,0.0,100,MSEL_01110-ICU
8,600000_0.5,600000,0.5,0,3,0.0,0.0,0.0,0.0,100,MSEL_01462
9,600000_0.5,600000,0.5,0,5,0.0,0.0,0.0,0.0,100,MSEL_01550


In [4]:
interictal_data = torch.load("data/interictal.pt")
preictal_data = torch.load("data/preictal.pt")
patient_ids = list(interictal_data.keys())

In [5]:
for test_patient in patient_ids:
    from models import FocalClassifier
    model_path = glob(f"experiments/focal_loss/meisel/{test_patient}/version_0/checkpoints/*.ckpt")[0]
    # print(model_path)
    model = FocalClassifier.load_from_checkpoint(model_path)
    segments = torch.load(f"data/test/{test_patient}_test_segments.pt")
    pred = model(segments)
    print(pred.max())

tensor(0.3624, grad_fn=<MaxBackward1>)
tensor(0.3636, grad_fn=<MaxBackward1>)
tensor(0.3703, grad_fn=<MaxBackward1>)
tensor(0.3503, grad_fn=<MaxBackward1>)
tensor(0.3723, grad_fn=<MaxBackward1>)
tensor(0.3744, grad_fn=<MaxBackward1>)
tensor(0.4009, grad_fn=<MaxBackward1>)
tensor(0.3471, grad_fn=<MaxBackward1>)
tensor(0.3846, grad_fn=<MaxBackward1>)
tensor(0.4298, grad_fn=<MaxBackward1>)
tensor(0.3858, grad_fn=<MaxBackward1>)
tensor(0.3871, grad_fn=<MaxBackward1>)
tensor(0.3613, grad_fn=<MaxBackward1>)
tensor(0.3731, grad_fn=<MaxBackward1>)
tensor(0.3989, grad_fn=<MaxBackward1>)
tensor(0.3765, grad_fn=<MaxBackward1>)
tensor(0.3946, grad_fn=<MaxBackward1>)
tensor(0.3861, grad_fn=<MaxBackward1>)
tensor(0.3834, grad_fn=<MaxBackward1>)
tensor(0.3600, grad_fn=<MaxBackward1>)
tensor(0.3739, grad_fn=<MaxBackward1>)
tensor(0.3758, grad_fn=<MaxBackward1>)
tensor(0.3839, grad_fn=<MaxBackward1>)


In [6]:
import torchviz

ModuleNotFoundError: No module named 'torchviz'

In [72]:
(pred > 0.5).sum()/len(pred)

tensor(0.0739)

tensor(0.4009, grad_fn=<MaxBackward1>)

In [74]:
8154*0.5/60

67.95

In [75]:
x = torch.load("data\interictal.pt")[test_patient]
y = torch.load("data\preictal.pt")[test_patient]

In [76]:
1 - model(x).sum()/len(x)

tensor(0.5214, grad_fn=<RsubBackward1>)

In [77]:
model(y).sum()/len(y)

tensor(0.4929, grad_fn=<DivBackward0>)